Sentiment Analysis in Stock Market

weiguang wang 	(NetID: wwg7031)

In this project, I focus on one stock of A-share market of China. To analyze investors’ sentiment, I prepare to crawl the comments of investors from the most famous stock BBS in China, called Oriental Wealth. After get the data, I will mark these data, to judge wether a given comments is positive or negative. Then did cross-validation, to check the result. Finally, analyze influence of predicted sentiments to the trend of the stock.

The first part of this project is crawling comments from the BBS. Therefore I wrote a web crawler showed as below to 
collect enough updated comments to do analyze.

In [ ]:
from bs4 import BeautifulSoup
import requests, csv

class commentsCrawler(object):
    def __init__(self):
        self.server = 'http://guba.eastmoney.com/list,300172'
        self.comments = []  # collect comments;         
    
    def get_contents(self):
        for i in range(20):
            target = self.server + '_'+str(1)+'.html'
            req = requests.get(url=target)
            html = req.text
            bf = BeautifulSoup(html, "lxml")
            texts = bf.find_all('span', class_ = 'l3') 
            for eachText in texts:
                a_bf = BeautifulSoup(str(eachText), "lxml")
                a = a_bf.find_all('a')
                for each in a:
                    self.comments.append(each.string)
    
    def writer(self):
        with open('comments.csv','w',newline='',encoding='utf-8-sig') as f:
            w = csv.writer(f)
            w.writerows(self.commments)

if __name__ == "__main__":
    comments = commentsCrawler()
    comments.get_contents()
    comments.writer()

Transfer and stored the original output file into 'comments.csv', and the file contains two column, one for comments, the other is manual marked sentiments, which defined in three different degree, positive, negative and neutral. 

The most traditional and easiest way to do the sentiment analysis is to count the number of positive and negative words in a sentence and determine whether it is denoted as positive or not. Although this method is quite easy to implement, it is not accuracy to present the real sentiment of a given sentence, for this way doesn't concern the relationship betweem two words in the sentence.  Another way called "bag of words" method, in this way, each of sentence are denoted as an N times 1 vector, where N represents the number of different words in the given sentence. In this way, we can use Logistic Regression or SuperVector Machine to build the model and use it to do sentiment analysis for a given sentence. However, "bag of words" method ignore the contextual information and the size of input data, which means in this way, we still hard to get good enough result.

After analysis the disadvantage of the two method mentioned above, in this project, I prepare to use RNN to try to solve this problem and to see the result of it. Based on the knowledge that we mentioned on the class, there are different types of RNN, and I plan to use the many-to-one RNN structure to do the sentiment analysis, which means the input of the RNN model are words in a given sentence and  the output is point the input get. 

In this project, I used some libraries to help me to build the model, like sklearn, gensim, keras. where sklearn is a machine learning library in python, gensim is library used to extract sematic topics in a given document, keras is also a library(or called deep learing APIs) of python. Before I train the model, I build the negative and positive training dataset first, creating "positive.xlsx" and "negative.xlsx". For the code, load all of the relevent library and data first.

In [1]:
# load libraries and initialize parameters used later
import yaml
import sys
from importlib import reload
reload(sys)

from sklearn.cross_validation import train_test_split
import multiprocessing
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import jieba
import pandas as pd
import sys

sys.setrecursionlimit(1000000)

# set parameters:
vocab_dim = 100
maxlen = 100
n_iterations = 1  
n_exposures = 10
window_size = 7
batch_size = 32
n_epoch = 4
input_length = 100
cpu_count = multiprocessing.cpu_count()

# load negative and positive comment data 
def loadfile():
    neg=pd.read_excel('../negative.xlsx',header=None,index=None)
    pos=pd.read_excel('../positive.xlsx',header=None,index=None)

    combined = np.concatenate((pos[0], neg[0]))
    y = np.concatenate((np.ones(len(pos),dtype=int), np.zeros(len(neg),dtype=int)))
    
    return combined,y

/Users/weiguang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.
/Users/weiguang/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


jieba is a library which can be used to segament chinese words. In this step, I did preprocessing again to convert each document to lower-case, remove the breaks for new lines and split on the whitespace finally.

In [2]:
def tokenizer(text):
    text = [jieba.lcut(document.replace('\n', '')) for document in text]
    return text

For most of circumstance in NLP, we should build "bag of words" first, at here, it means that we have to build a dictionary combined word embedding and word index for each of word, and correspoding words index in each sentence. For there is a tight relationship between semantic properties of each word and context of each sentence, we can not just consider the words in sentence individually. The most common ways to keep the relationship of each word is using vector space model to represent a sentence into word embedding. 

In [5]:
#Creates a word to index mapping
#Creates a word to vector mapping
#Transforms the Training and Testing Dictionaries

def create_dictionaries(model = None,
                        combined = None):
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()} 
        w2vec = {word: model[word] for word in w2indx.keys()}

        def parse_dataset(combined):
            data = []
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)
                data.append(new_txt)
            return data
        combined = parse_dataset(combined)
        combined = sequence.pad_sequences(combined, maxlen=maxlen)
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')

def word2vec_train(combined):
    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined)
    model.train(combined, total_examples=500, epochs=n_epoch)
    model.save('Word2vec_model.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined

In this project, I define that if the frequence of word is lower than 10, then the index of the word will be marked as zero, otherwise, it will be marked as the corresponding frequency. To identify the accuracy of the model, I seperate the input data(negative and positive seperately) into two parts, one is used to train the LSTM model, and the other is used to do the test, to get the accuracy of the model.

In [6]:
def get_data(index_dict,word_vectors,combined,y):
    n_symbols = len(index_dict) + 1  
    embedding_weights = np.zeros((n_symbols, vocab_dim))
    for word, index in index_dict.items():
        embedding_weights[index, :] = word_vectors[word]
    x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2)
    print (x_train.shape,y_train.shape)
    return n_symbols,embedding_weights,x_train,y_train,x_test,y_test

After get the word embedding and word indexs, in this project, I used the LSTM to build the prediction model. By using the LSTM model, at here, I used the sigmoid function as the activation function, and use Stochastic gradient descent as the optimizer of the model.

In [8]:
def train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test):
    model = Sequential()
    
    model.add(Embedding(output_dim=vocab_dim, input_dim=n_symbols,
                        mask_zero=True, weights=[embedding_weights],
                        input_length=input_length))  
    
    model.add(LSTM(output_dim=50, activation='sigmoid', inner_activation='hard_sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='SGD',metrics=['accuracy'])

    print ("Train...")
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=n_epoch,verbose=1, validation_data=(x_test, y_test))

    print ("Evaluate...")
    score = model.evaluate(x_test, y_test,
                                batch_size=batch_size)

    yaml_string = model.to_yaml()
    with open('lstm.yml', 'w') as outfile:
        outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
    model.save_weights('lstm.h5')
    print ('Test score:', score)

By running the model, we can see that, the accuracy of positive part is  and the accuracy of negative part is . This may caused by 

In [10]:
combined,y=loadfile()
print (len(combined),len(y))
combined = tokenizer(combined)
index_dict, word_vectors, combined=word2vec_train(combined)
n_symbols,embedding_weights,x_train,y_train,x_test,y_test=get_data(index_dict, word_vectors,combined,y)
train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test)

FileNotFoundError: [Errno 2] No such file or directory: 'negative.xlsx'

In [ ]:
def input_transform(string):
    words=jieba.lcut(string)
    words=np.array(words).reshape(1,-1)
    model=Word2Vec.load('Word2vec_model.pkl')
    _,_,combined=create_dictionaries(model,words)
    return combined

def lstm_predict(string):
    print ('loading model......')
    with open('lstm.yml', 'r') as f:
        yaml_string = yaml.load(f)
    model = model_from_yaml(yaml_string)

    print ('loading weights......')
    model.load_weights('lstm.h5')
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    data=input_transform(string)
    data.reshape(1,-1)
    #print data
    result=model.predict_classes(data)
    if result[0][0]==1:
        print (string,' positive')
    else:
        print (string,' negative')

if __name__=='__main__':
    testData = ["上影线告诉你  我要开始跌了", "狗庄是不是死光了", "环保风暴来临","大涨！！！"]
    testDataPoints = ["negative","negative","positive","positive"]; # this is the result we hope to get.
    for string in testData:
        lstm_predict(string)